In [1]:
from __future__ import absolute_import, division, print_function

import sys
sys.path.append("TensorFlow20_ResNet")
import tensorflow as tf
from models.resnet import resnet_18, resnet_34, resnet_50, resnet_101, resnet_152
import config
from prepare_data import generate_datasets

import math
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
import scipy
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Model configuration
img_width, img_height, img_num_channels = 32, 32, 3
no_epochs = 100
optimizer = Adam()
validation_split = 0.2
verbosity = 1

# Load CIFAR-10 data
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

X_train.shape
input_train = X_train
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = X_train .astype('float32')
input_test = X_test.astype('float32')

# Normalize data
X_train = (X_train / 255).astype(np.float32)
X_test = (X_test / 255).astype(np.float32)

In [3]:
model = resnet_34()
model.build(input_shape=(None, 32, 32, 3))

<h3>Training for minimizing the Rotation Loss</h3>

In [11]:
lr = 1e-4
e = 1
m = X_train.shape[0]
batch_size = 128

while True:
    e+=1
    batch_count = 0
#     for idx in tqdm(range(0, m, batch_size)): # batch
    for idx in range(0, m, batch_size): # batch
        images = tf.convert_to_tensor(X_train[idx:idx+batch_size, ...])
        
        degrees_to_rotate = np.random.choice([90, 180, 270]) # randomly choosing the rotation angle
        
        images_rotated = tf.convert_to_tensor(scipy.ndimage.rotate(images, degrees_to_rotate, axes=(2, 1))) 

        with tf.GradientTape() as tape:
            embs_a, embs_b = model(images), model(images_rotated)
#             loss = tf.norm(embs_a-embs_b, ord='euclidean', axis=None, keepdims=None, name=None)
            loss = tf.reduce_sum(tf.norm(embs_a-embs_b, ord='euclidean', axis=(1), keepdims=None, name=None))

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer = tf.keras.optimizers.Adam(lr=lr)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if batch_count%5==0:
            print(f"\tbatch {batch_count}, loss: {tf.reduce_sum(loss).numpy()}")
        batch_count+=1
    if e%1 == 0:
        clear_output()
        print(f"epoch {e}, loss: {loss}")

	batch 0, loss: 16.42196273803711
	batch 5, loss: 7.798646926879883
	batch 10, loss: 4.542409420013428
	batch 15, loss: 2.789872884750366
	batch 20, loss: 1.6361074447631836
	batch 25, loss: 1.1954256296157837


SystemError: <built-in function TF_NewBuffer> returned a result with an error set